<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/hor_disclosures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF Primer

In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
import io
import requests
import re
import pandas as pd

from PyPDF2 import PdfReader
import requests
import zipfile
import xml.etree.ElementTree as ET
import os
# Parallelize.
import io
import xml.etree.ElementTree as ET
import aiohttp
import asyncio

import pandas as pd
from bs4 import BeautifulSoup
from typing import Any, List, Optional


BASE_URL = "https://disclosures-clerk.house.gov/public_disc/financial-pdfs/"
FINANCIAL_DOC_URL= "https://disclosures-clerk.house.gov/public_disc/ptr-pdfs"

## Starting point. Reading year's financial disclosures

In [ ]:
def extract_xml_from_zip_url(url, output_file):
    """Extracts an XML file from a ZIP archive available at a given URL and saves it to a specified file.

    Args:
        url: The URL of the ZIP archive.
        output_file: The path to save the extracted XML file.
    """
    try:
        # Download the ZIP file
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(f"Failed to download ZIP file: {response.status_code}")

        # Create a BytesIO object for the ZIP file
        zip_file = io.BytesIO(response.content)

        # Extract the XML file from the ZIP
        with zipfile.ZipFile(zip_file, "r") as zip_ref:
            for member in zip_ref.infolist():
                if member.filename.endswith(".xml"):
                    xml_stream = io.BytesIO(zip_ref.read(member))
                    break

        return xml_stream
    except Exception as e:
        print(f"Error extracting XML file: {e}")




## Extract Disclosures Doc Ids

In [ ]:
# Assuming you have a BytesIO object named 'xml_data' containing the XML
def extract_docids_from_year_disclosures(res):
  xml_data = res
  # Parse the XML data from the BytesIO object
  tree = ET.parse(xml_data)
  root = tree.getroot()

  # Find all members with FilingType == "P" and extract their DocID
  doc_ids = []
  for member in root.findall('Member'):
      filing_type = member.find('FilingType').text
      if filing_type == "P":
          doc_id = member.find('DocID').text
          membername = f"{member.find('Last').text} {member.find('First').text}"
          state = member.find('StateDst').text
          filing_date = member.find('FilingDate').text
          doc_ids.append(dict(doc_id=doc_id, member=membername, state=state, filing_date=filing_date))
      else:
        pass#print(f'Found filing type :{filing_type}') # Need to find all filing types # and place by filing_date. perhaps in a df
  return doc_ids

In [ ]:
def read_pdf_from_url(url): # need asyncio client here
  headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}
  response = requests.get(url=url, headers=headers, timeout=120)#
  on_fly_mem_obj = io.BytesIO(response.content)
  pdf_reader = PdfReader(on_fly_mem_obj)
  return pdf_reader


In [ ]:
def extract_filer(lines):
  transaction_dict = {}
  for line in lines[2:]:
    if ':' in line:
      key, value = line.split(':', 1)
      transaction_dict[key.strip()] = value.strip()


def extract_from_disclosure(pdf_reader):
  holder = {}
  # page 0 has all information about the HOR Member
  start_token = 'SP' # START token
  # then we have two blanks
  # then wee have ticker [ST] Transaction Transaction Date Amount #

  # we actually dont need filer info as all the information is already in the xml file
  # we should extract it from there. we should not even need the dates then
  '''
  <Member>
    <Prefix/>
    <Last>Abolfazli</Last>
    <First>Maryam.</First>
    <Suffix/>
    <FilingType>C</FilingType>
    <StateDst>TN05</StateDst>
    <Year>2024</Year>
    <FilingDate>5/13/2024</FilingDate>
    <DocID>10060479</DocID>
    </Member>
  </Member>
  '''
  filer_info = {}
  for page_num in range(0, len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]

      text = page.extract_text()
      if text.find('Clerk of the House of Representatives') > 0:
        start_filer_section = text.index('Clerk of the House of Representatives')
        start_transaction_section = text.index('ID')
        filer = text[start_filer_section:start_transaction_section].encode('utf-8').decode('utf-8')
        transactions = text[start_transaction_section +12: ]
        lines = filer.splitlines()
        filer_info = lines[2:]
      else:
        transactions = text

      rowHolder = []
      rows = transactions.splitlines()
      seen = False
      transaction_holder = []
      for row in rows:
          row  = row.encode('utf-8').decode('utf-8')
          if row.startswith(start_token):
            seen = !seen
          if seen:
            if row:
              if row.find("\x00") <0:
                transaction_holder.append(row)
              else:
                seen = False
                rowHolder.append(' '.join(transaction_holder))
                transaction_holder = []
      holder[page_num] = rowHolder


  return {'info' : filer_info, 'transactions' : holder}

def extract_data(first_row):
  import re
  ticker_idx_st = first_row.find('(')

  if ticker_idx_st < 0:
    ticker = 'N/A'
    company_name = first_row[0:first_row.find('[')]
  else:
    ticker_idx_end = first_row.index(')')
    ticker = first_row[ticker_idx_st+1:ticker_idx_end]
    company_name = first_row[0:ticker_idx_st]

  if first_row.find('[') < 0:
    action = 'N/A'
    purchase_price = 'N/A'
  else:
    action_idx = first_row.index('[') + 4
    purchase_idx = first_row.index('$')
    action = first_row[action_idx:action_idx+2]
    purchase_price = first_row[purchase_idx:]


  pattern = r"\d{2}/\d{2}/\d{4}"
  matches = re.findall(pattern, first_row)

  # Extract the two dates from the matches
  dates = matches
  if dates:
    date1 = dates[0]
    date2 = dates[1] if len(dates) > 1 else 'N/A'
  else:
    date1 = 'N/A'
    date2 = 'N/A'

  # need to add filer info
  data = dict(company=company_name, ticker=ticker, action=action, purchase_price=purchase_price, transaction_date=date1,
            report_date=date2)
  return data


def extract_transactions(holder_dict, member_dict):
  row_list = []
  for page_num, lst in holder_dict.items():
    for row in lst:
      row = extract_data(row)
      row.update(member_dict)
      row_list.append(row)

  return row_list


### Starting point

In [ ]:
### Logic#
# Sample PDF
#def extract_financial_disclosures(financial_doc_url):
#  pdf_reader = read_pdf_from_url(financial_doc_url)
#  transactions = extract_transactions(pdf_reader)
#  return transactions

def disclosures_report(disclosure_tpl):
  url, ddict = disclosure_tpl
  pdf_reader = read_pdf_from_url(url) # need asyncio client to make request
  data = extract_from_disclosure(pdf_reader)
  dfdata = extract_transactions(data['transactions'], ddict)
  if dfdata:
    return pd.DataFrame(data=dfdata )
  return pd.DataFrame()


def disclosures_reports(financial_doc_urls):
  all_dfs = []
  for url in financial_doc_urls:
    dfdata = disclosures_report(url)
    if dfdata:
      all_dfs.append(pd.DataFrame(data=dfdata ))
    #from pprint import pprint
    #pprint(transactions)
  return pd.concat(all_dfs, ignore_index=True)

def get_report_for_year(year):
  zip_name = f'{year}FD.zip'
  url = f"{BASE_URL}/{zip_name}"
  output_file = "2024FD.xml"
  res = extract_xml_from_zip_url(url, output_file)
  ### Extract all the doc_ids
  return extract_docids_from_year_disclosures(res)

def fetch_all_disclosures(year):

  # all operationsbeowasync task to fetch dat and extract xml
  disclosure_dicts = get_report_for_year(year) # async

  financial_doc_urls_tpls = [(f"{FINANCIAL_DOC_URL}/{year}/{ddict['doc_id']}.pdf", ddict) for ddict in disclosure_dicts]

  disclosures = [disclosures_report(financial_doc_url_tpl) for financial_doc_url_tpl in financial_doc_urls_tpls]

  all_transactions_df = pd.concat(disclosures, ignore_index=True)

  return all_transactions_df

  # we need to create tasks for each financial doc urls
  '''
  tasks = [disclosures_for_doc(client, docid) for docidrow in financial_doc_urls]
  all_transactions = await asyncio.gather(*tasks)
  all_transactions_df = pd.concat(all_transactions, ignore_index=True)
  return all_transactions_df
  '''




In [ ]:
fetch_all_disclosures('2024')


#### ASYNC VERSION

In [ ]:
def extract_docids_from_year_disclosures(res):
  xml_data = res
  # Parse the XML data from the BytesIO object
  tree = ET.parse(xml_data)
  root = tree.getroot()

  # Find all members with FilingType == "P" and extract their DocID
  doc_ids = []
  for member in root.findall('Member'):
      filing_type = member.find('FilingType').text
      if filing_type == "P":
          doc_id = member.find('DocID').text
          membername = f"{member.find('Last').text} {member.find('First').text}"
          state = member.find('StateDst').text
          filing_date = member.find('FilingDate').text
          doc_ids.append(dict(doc_id=doc_id, member=membername, state=state, filing_date=filing_date))
      else:
        pass#print(f'Found filing type :{filing_type}') # Need to find all filing types # and place by filing_date. perhaps in a df
  return doc_ids


async def aextract_xml_from_zip_url(client, url, output_file):
    """Extracts an XML file from a ZIP archive available at a given URL and saves it to a specified file.

    Args:
        url: The URL of the ZIP archive.
        output_file: The path to save the extracted XML file.
    """
    async with client.get(url) as response:
        # Download the ZIP file
        if response.status == 200:
          content = b""
          while True:
              data = await response.content.read(1024)  # Read up to 1024 bytes
              if not data:
                  break
              content += data
    zip_file = io.BytesIO(content)
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
      for member in zip_ref.infolist():
          if member.filename.endswith(".xml"):
              xml_stream = io.BytesIO(zip_ref.read(member))
              break
    return extract_docids_from_year_disclosures(xml_stream)


async def aread_pdf_from_url(client, discl_dict): # need asyncio client here

  disclosure_url = f"{FINANCIAL_DOC_URL}/{year}/{discl_dict['doc_id']}.pdf"
  headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

  async with client.get(disclosure_url, headers=headers, timeout=120) as response:
    if response.status == 200:
          content = b""
          while True:
              data = await response.content.read(1024)  # Read up to 1024 bytes
              if not data:
                  break
              content += data

  on_fly_mem_obj = io.BytesIO(content)
  pdf_reader = PdfReader(on_fly_mem_obj)

  data = extract_from_disclosure(pdf_reader)
  dfdata = extract_transactions(data['transactions'], discl_dict)
  if dfdata:
    return pd.DataFrame(data=dfdata )
  return pd.DataFrame()


async def afetch_all_transactions(session, reports):
    tasks = [aread_pdf_from_url(session, report) for report in reports]
    all_transactions = await asyncio.gather(*tasks)
    all_transactions_df = pd.concat(all_transactions, ignore_index=True)
    return all_transactions_df




In [ ]:
year = '2024'
zip_name = f'{year}FD.zip'
url = f"{BASE_URL}/{zip_name}"
output_file = f"{year}.xml"
session = aiohttp.ClientSession()


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fa1a3be6050>


In [ ]:
reports = await aextract_xml_from_zip_url(session, url, output_file)

In [ ]:
hor_disclosures = await afetch_all_transactions(session, reports)


In [ ]:
hor_disclosures

,company,ticker,action,purchase_price,transaction_date,report_date,doc_id,member,state,filing_date
0,SP Albemarle Corporation,ALB,S,"$1,001 - $15,000",12/21/2023,01/08/2024,20024277,Allen Richard W.,GA12,1/11/2024
1,SP Albemarle Corporation,ALB,S,"$1,001 - $15,000",12/21/2023,01/08/2024,20024277,Allen Richard W.,GA12,1/11/2024
2,SP Charles Schwab Corporation,SCHW,P,"$50,001 - $100,000",12/14/2023,01/08/2024,20024277,Allen Richard W.,GA12,1/11/2024
3,"SP NextEra Energy, Inc.",NEE,S,"$15,001 - $50,000",12/21/2023,01/08/2024,20024277,Allen Richard W.,GA12,1/11/2024
4,"SP BlackRock, Inc. Common Stock",BLK,S,"$50,001 - $100,000",01/17/2024,02/05/2024,20024458,Allen Richard W.,GA12,2/16/2024
...,...,...,...,...,...,...,...,...,...,...
486,SP Stellantis N.V. Common Shares,STLA,P,"$1,001 - $15,000",06/30/2021,05/30/2024,20025151,Williams Roger,TX25,6/12/2024
487,SP Sunshine Biopharma Inc. - Common stock,SBFM,P,"$1,001 - $15,000",06/30/2022,05/30/2024,20025151,Williams Roger,TX25,6/12/2024
488,SP Truist Financial Corporation Common Stock,TFC,P,"$1,001 - $15,000",06/30/2022,05/30/2024,20025151,Williams Roger,TX25,6/12/2024
489,SP Walt Disney Company,DIS,S,"$1,001 - $15,000",06/22/2022,05/30/2024,20025151,Williams Roger,TX25,6/12/2024
